In [ ]:
%matplotlib inline

from elasticsearch import Elasticsearch

es = Elasticsearch([{'host':'cl-analytics.mwt2.org', 'port':9200}],
                   send_get_body_as = 'POST'     # to be passed to Transport class
                  )

triumf = ["ANALY_TRIUMF",
          "TRIUMF",
          "TRIUMF_HIMEM",
          "TRIUMF_MCORE",
          "TRIUMF_MCORE_LOMEM"
         ]

q = {
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "query_string": {
            "query": "computingsite:*TRIUMF* AND jobstatus:f*",
            "analyze_wildcard": true,
            "lowercase_expanded_terms": false
          }
        },
        {
          "range": {
            "endtime": {
              "gte": "now-30d/d",
              "lte": "now/d"
            }
          }
        }
      ],
      "must_not": []
    }
  },
  "aggs": {
    "6": {
      "date_histogram": {
        "field": "endtime",
        "interval": "1d",
        "time_zone": "UTC",
        "min_doc_count": 1
      },
      "aggs": {
        "7": {
          "terms": {
            "field": "computingsite",
            "size": 10,
            "order": {
              "_term": "asc"
            }
          },
          "aggs": {
            "cpuTimeH": {
              "sum": {
                "script": {
                  "inline": "doc['cpuconsumptiontime'].value/3600",
                  "lang": "expression"
                }
              }
            },
            "wallTimeH": {
              "sum": {
                "script": {
                  "inline": "doc['wall_time'].value/3600",
                  "lang": "expression"
                }
              }
            },
            "stageInTimeH": {
              "sum": {
                "script": {
                  "inline": "doc['timeStageIn'].value/3600",
                  "lang": "expression"
                }
              }
            }
          }
        }
      }
    }
  }
}

